In [7]:
!pwd

/home/eldor/personal_projects/guessing-game-with-robot


In [ ]:
# combine
### reset ###
from IPython import get_ipython
ipython = get_ipython()
ipython.magic('reset -sf') 
import gc
gc.collect()
print("Notebook environment has been reset.")

### main code ###
import numpy as np
import mujoco
import mediapy as media
from scripts.interface import SimulatedRobot

# Load the model and initialize data
xml = 'robot_control/mujoco/low_cost_robot/scene.xml'
model = mujoco.MjModel.from_xml_path(xml)
data = mujoco.MjData(model)

# Set rendering parameters
renderer = mujoco.Renderer(model, height=480, width=640)
camera = mujoco.MjvCamera()
mujoco.mjv_defaultFreeCamera(model, camera)
camera.distance = 0.5

# User choice: Set initial pose explicitly or use the current state
USE_CUSTOM_INITIAL_POSE = True  # Change to False to use automatic initial pose

if USE_CUSTOM_INITIAL_POSE:
    # Specify initial pose
    # initial_qpos = [0.0, 0.0, 0.0, np.pi / 4, -np.pi / 4, 0.0]
    initial_qpos = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    data.qpos[:6] = initial_qpos
    mujoco.mj_forward(model, data)  # Apply the initial pose
    print("Initial pose (specified):", initial_qpos)
else:
    # Use current state as initial pose
    mujoco.mj_forward(model, data)  # Initialize the simulation state
    print("Initial pose (automatic): Current state used.")

# Get initial position
jname = 'joint6'
body_id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_BODY, jname)
initial_position = data.geom_xpos[body_id]
print("Initial position:", initial_position)

# Target position
target_point = [0.1, 0.15, 0.3]
target_ori = np.identity(3)

# Initialize simulated robot
r = SimulatedRobot(model, data)

# IK Loop
frames = []
fps = 30
adjustment_rate = 0.2
max_steps = 100
prev_error = np.inf

for i in range(max_steps):
    # Solve IK for the target position
    q_target_pos = r.solve_ik(target_point)
    r.d.qpos[:6] += adjustment_rate * (q_target_pos[:6] - r.d.qpos[:6])
    
    # printing current joint positions
    # print(r.d.qpos)

    # Update physics and rendering
    mujoco.mj_forward(r.m, r.d)
    renderer.update_scene(r.d, camera)
    frames.append(renderer.render())

    # Calculate error
    current_pos = r.d.geom_xpos[body_id]
    error = np.linalg.norm(target_point - current_pos)

    # Check for convergence
    if abs(error - prev_error) <= 1e-5:
        print(f"Converged at step {i} with error: {error}")
        break

    prev_error = error

# Final position
print("Final position:", current_pos)
media.show_video(frames, fps=fps, loop=False)


/tmp/ipykernel_3806326/3425466801.py:5: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  ipython.magic('reset -sf')


Notebook environment has been reset.
Initial pose (specified): [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Initial position: [0.0135 0.2845 0.1665]
Final position: [0.14404657 0.16452056 0.30174929]


This browser does not support the video tag.
